In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install tf2cv tensorflow>=2.0.0

In [3]:
from tensorflow.python.client import device_lib
dev=device_lib.list_local_devices()
dev[len(dev)-1]

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9877871749680092866
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"

In [0]:
#global consts
IMG_SIZE='137x236'
HEIGHT = 137
WIDTH = 236
BATCH_SIZE=128
DROP_OUT=0.0
FOLD=2
TRAIN_DIR = './train/'
TRAIN_IMG_DIR=TRAIN_DIR+str(IMG_SIZE)
EPOCHS = 200
STATS_FS = (0.055029564364430086, 0.17228061284674265)
STATS_128 = (0.0692, 0.2051)
EXPERIMENT='sernext_fullsize_5fold_f2'
INPUT_DIR ='/content/drive/My Drive/kaggle/bengali/input/'
OUTPUT_DIR = '/content/drive/My Drive/kaggle/bengali/output/'
IMPORT_DIR = '/content/drive/My Drive/kaggle/bengali/nbs/'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'
CHKP_DIR = EXPERIMENT_DIR+'chkp/'
OUTPUT_SUBDIR = ''
DATASET_224='224x224-bengali.zip'
DATASET_128='grapheme-imgs-128x128.zip'
DATASET_FS = 'bengali-full-size.zip'
TRAIN_WITH_FOLD = 'train_with_fold.csv'
PRETRAINED = False
PRETRAINED_WEIGHTS = ''
LOG_FILE = EXPERIMENT_DIR+'log_'+EXPERIMENT+'.csv'
#INITIAL_EPOCH=40
#HEAD_WD=1e-3
#BACKBONE_WD = 1e-4

if IMG_SIZE == 128: DATASET=DATASET_128
if IMG_SIZE == 224: DATASET=DATASET_224
if IMG_SIZE == '137x236' : DATASET=DATASET_FS


#lr sheduler consts
LR_MAX = 0.001
LR_MIN = 0.00001
T_MAX = 100

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append(IMPORT_DIR+'py/models/')
sys.path.append(IMPORT_DIR+'py/utils/')
sys.path.append(IMPORT_DIR+'py/callbacks/')
sys.path.append(IMPORT_DIR+'py/opts/')

In [0]:
#import local modules
#from mish import Mish,mish
from generators import *
from kaggle_metric import *
from cosine import CosineAnnealingScheduler
from kagglevalidation import KaggleValidationFS
#from lookahead import Lookahead
from se_resnext50 import build_se_resnext50_model_fs_wod
#from md121v2 import build_md121_v2_model
#from losses import categorical_focal_loss


#import external modules
import numpy as np 
import pandas as pd 

import os
import errno

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

In [8]:
try:
  os.mkdir(TRAIN_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_DIR+' already exists')
    else:
        raise
try:
  os.mkdir(TRAIN_IMG_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(TRAIN_IMG_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(EXPERIMENT_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(EXPERIMENT_DIR+' already exists')
    else:
        raise

try:
  os.mkdir(CHKP_DIR)
except OSError as e:
    if e.errno == errno.EEXIST:
        print(CHKP_DIR+' already exists')
    else:
        raise


os.system('cp '+ '"'+INPUT_DIR+DATASET+'" ' + TRAIN_DIR)
os.system('cp '+ '"'+INPUT_DIR+TRAIN_WITH_FOLD+'" ' + TRAIN_DIR)

/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/ already exists
/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ already exists


0

In [9]:
os.system('unzip -q '+TRAIN_DIR+DATASET+ ' -d '+ TRAIN_IMG_DIR)

0

In [10]:
!ls train

137x236  bengali-full-size.zip	train_with_fold.csv


In [0]:
dataset_np = pd.read_csv(TRAIN_DIR+TRAIN_WITH_FOLD).to_numpy()
dataset_np[:,0]+='.png'
dataset_m = dataset_np.shape[0]


valid_m = np.where(dataset_np[:,6]==FOLD)[0].shape[0]
train_m = dataset_m-valid_m

fold_train_inds = np.where(dataset_np[:,6] != FOLD)
fold_valid_inds = np.where(dataset_np[:,6] == FOLD)

train_np = dataset_np[fold_train_inds]
valid_np = dataset_np[fold_valid_inds]

train_df = pd.DataFrame(train_np)
valid_df = pd.DataFrame(valid_np)

train_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)
valid_df.rename(columns={0:'filename',1:'root_class',2:'vowel_class',3:'cons_class', 4:'grapheme'}, inplace=True)

train_df.drop(columns=[5,6], inplace=True)
valid_df.drop(columns=[5,6], inplace=True)

In [0]:
model =  build_se_resnext50_model_fs_wod(height=HEIGHT, width=WIDTH)

In [0]:
cutout_datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(p=0.95,s_l=0.05,s_h=0.3,r_1=0.8,r_2=1.2,v_l=0,v_h=0))

mcm_datagen = ImageDataGenerator()

In [14]:
columns=["root_class","vowel_class", "cons_class"]

mcm_generator1 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

mcm_generator2 = mcm_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")


cutout_generator = cutout_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=TRAIN_IMG_DIR,
        x_col="filename",
        y_col=columns,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="multi_output",
        color_mode="grayscale")

Found 160672 validated image filenames.
Found 160672 validated image filenames.
Found 160672 validated image filenames.


In [0]:
def g3_generator_fs2(gen1,gen2, gen3,h=137,w=236,st=STATS_FS):
    while True:
        p = np.random.rand()
        if p>0.8:
            batch = next(gen3)
            x_res,y_res = prep_batch_fs(batch,st=st)
        else:
            batch1 = next(gen1)
            batch2 = next(gen2)
            q = np.random.rand()
            if q>0.3:
                x_res,y_res = mix_batches_fs(batch1,batch2,alpha=0.1,st=st)
            else:
                x_res,y_res = cutmix_batches_fs2(batch1,batch2,alpha=0.8,h=h,w=w,st=st)
        yield x_res,y_res

In [0]:
g3_datagen_fs2 = g3_generator_fs2(mcm_generator1,mcm_generator2,cutout_generator,h=HEIGHT,w=WIDTH, st=STATS_FS)

In [0]:
if 0: 
  batch_x,y = next(g3_datagen_fs2)
  import matplotlib.pyplot as plt
  fig, axs = plt.subplots(1, 7, figsize=(30, 30))
  for i in range(7):
    axs[i].imshow(batch_x[i,:,:,0], cmap='gray')
  plt.show()

In [0]:
opt=tf.keras.optimizers.Adam(learning_rate=LR_MAX)

In [0]:
checkpoint =  tf.train.Checkpoint(latest_epoch=tf.Variable(0), optimizer=opt, model=model)
chkp_manager =  tf.train.CheckpointManager(checkpoint, CHKP_DIR, max_to_keep=3)

In [20]:
chkp_manager.latest_checkpoint

'/content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-84'

In [21]:
chkp_restore_status=checkpoint.restore(chkp_manager.latest_checkpoint)
if chkp_manager.latest_checkpoint:
  #chkp_restore_status.assert_consumed()
  print("Restored from {}".format(chkp_manager.latest_checkpoint))
  print(f"Lastest epoch {checkpoint.latest_epoch.numpy()}")
else:
  print("Initializing from scratch.")

Restored from /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-84
Lastest epoch 84


In [0]:
loss_dict={'root':    'categorical_crossentropy',
           'vowel':    'categorical_crossentropy',
           'consonant':'categorical_crossentropy'}
model.compile(optimizer=opt, loss=loss_dict, loss_weights=[4.0,1.0,1.0])

In [0]:
initial_epoch = checkpoint.latest_epoch.numpy()

print(f'epochs trained: {initial_epoch}. start training epoch {initial_epoch+1}...')
#ca_shed = CosineAnnealingScheduler(T_max=100, eta_max=0.0001, eta_min=0.000001,verbose=1, initial_epoch=59)
rolrplt = tf.keras.callbacks.ReduceLROnPlateau(monitor='kaggle',mode='max', patience=3, verbose=1, factor=0.5, min_lr=LR_MIN)


suffix =EXPERIMENT+\
                '_eps'+str(EPOCHS)+\
                '_iniep'+str(initial_epoch)+\
                '_sz'+str(IMG_SIZE)+\
                '_bs'+str(BATCH_SIZE)+\
                '_do'+str(DROP_OUT)+\
                '_'+model.optimizer.get_config()['name']+\
                '_IS_fold'+str(FOLD)


kv = KaggleValidationFS(valid_df,
                      batch_size=512,
                      vals_per_epoch=1,
                      train_steps=train_m//BATCH_SIZE+1,
                      logfile=EXPERIMENT_DIR+'log_kr_'+suffix+'.csv',
                      initial_epoch=initial_epoch,
                      suffix = suffix,
                      train_img_dir=TRAIN_IMG_DIR,
                      output_dir=EXPERIMENT_DIR,
                      thresholds=[0.987,0.995,0.994],
                      chkp_manager=chkp_manager,
                      in_shape=[HEIGHT,WIDTH],
                      st=STATS_FS)

#tf.keras.backend.set_value(model.optimizer.lr, 0.01)
cbs = [kv,rolrplt]

history=model.fit(g3_datagen_fs2,
                  epochs=EPOCHS,
                  steps_per_epoch=train_m//BATCH_SIZE+1,
                  callbacks=cbs,
                  verbose=2)

epochs trained: 84. start training epoch 85...
Train for 1256 steps
Epoch 1/200

batch 1255: validation...



[0:21:33] - kaggle:0.9808 - root:0.9742 -vowel:0.9868 - cons: 0.9878

saved checkpoint for epoch 85: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-85

1256/1256 - 1296s - loss: 3.4059 - root_loss: 0.6917 - vowel_loss: 0.3928 - consonant_loss: 0.2464
Epoch 2/200

batch 1255: validation...



[0:42:37] - kaggle:0.9807 - root:0.9750 -vowel:0.9870 - cons: 0.9857

saved checkpoint for epoch 86: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-86

1256/1256 - 1264s - loss: 3.3972 - root_loss: 0.6904 - vowel_loss: 0.3903 - consonant_loss: 0.2453
Epoch 3/200

batch 1255: validation...



[1:03:34] - kaggle:0.9816 - root:0.9759 -vowel:0.9880 - cons: 0.9867

saved checkpoint for epoch 87: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-87

1256/1256 - 1257s - loss: 3.3721 - root_loss: 0.6852 - vowel_loss: 0.3862 - consonant_loss: 0.2451
Epoch 4/200

batch 1255: validation...



[1:24:31] - kaggle:0.9826 - root:0.9770 -vowel:0.9876 - cons: 0.9889

saved checkpoint for epoch 88: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-88

1256/1256 - 1257s - loss: 3.4581 - root_loss: 0.7029 - vowel_loss: 0.3960 - consonant_loss: 0.2507
Epoch 5/200

batch 1255: validation...


...


[5:35:57] - kaggle:0.9854 - root:0.9820 -vowel:0.9893 - cons: 0.9885

saved checkpoint for epoch 100: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-100

1256/1256 - 1257s - loss: 3.0054 - root_loss: 0.6060 - vowel_loss: 0.3564 - consonant_loss: 0.2249
Epoch 17/200

batch 1255: validation...



[5:56:54] - kaggle:0.9834 - root:0.9792 -vowel:0.9895 - cons: 0.9856

saved checkpoint for epoch 101: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-101

1256/1256 - 1257s - loss: 3.1852 - root_loss: 0.6422 - vowel_loss: 0.3777 - consonant_loss: 0.2387
Epoch 18/200

batch 1255: validation...



[6:17:51] - kaggle:0.9853 - root:0.9813 -vowel:0.9900 - cons: 0.9886

saved checkpoint for epoch 102: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-102

1256/1256 - 1257s - loss: 3.0081 - root_loss: 0.6063 - vowel_loss: 0.3559 - consonant_loss: 0.2269
Epoch 19/200

batch 1255: validation...



[6:38:50] - kaggle:0.9841 - root:0.9800 -vowel:0.9885 - cons: 0.9880

saved checkpoint for epoch 103: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-103


Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
1256/1256 - 1259s - loss: 3.1236 - root_loss: 0.6298 - vowel_loss: 0.3702 - consonant_loss: 0.2340
Epoch 20/200

batch 1255: validation...



[6:59:49] - kaggle:0.9850 - root:0.9815 -vowel:0.9888 - cons: 0.9880

saved checkpoint for epoch 104: /content/drive/My Drive/kaggle/bengali/output/sernext_fullsize_5fold_f2/chkp/ckpt-104

1256/1256 - 1258s - loss: 2.9041 - root_loss: 0.5846 - vowel_loss: 0.3463 - consonant_loss: 0.2195
Epoch 21/200
